<a href="https://colab.research.google.com/github/Irfan1401/D121231093_pemrograman-web/blob/main/timm_tf_efficientnet_b0_in1k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hubungkan dengan google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import library training

In [ ]:
!pip install --upgrade keras keras-cv -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 650.7/650.7 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 43.9 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
import keras_cv
import numpy as np
import os

import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

# Muat Datasetnya

Setel Direktori

In [ ]:
BASE_DIR = '/content/drive/MyDrive/Bahan AI/Image Classification/Data'
TRAIN_DIR = os.path.join(BASE_DIR, 'Training')
TEST_DIR = os.path.join(BASE_DIR, 'Testing')

Parameter

In [ ]:
BATCH_SIZE = 32
IMG_SIZE = (224, 224)

Tentukan data training

In [ ]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    validation_split=0.2, # 20% data training akan jadi data validasi
    subset="training",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

Found 2870 files belonging to 4 classes.
Using 2296 files for training.


Tentukan data validasi

In [ ]:
validation_dataset = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

Found 2870 files belonging to 4 classes.
Using 574 files for validation.


Tentukan data testing

In [ ]:
test_dataset = tf.keras.utils.image_dataset_from_directory(
    TEST_DIR,
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical',
    shuffle=False #DATA JANGAN DIACAK SAAT EVALUASI
)

Found 394 files belonging to 4 classes.


Ambil nama kelas dari dataset (akan sama untuk semua)

In [ ]:
class_names = train_dataset.class_names
num_classes = len(class_names) # 4 kelas
print("Kelas yang terdeteksi:", class_names)
print(f"Jumlah kelas: {len(class_names)}")

Kelas yang terdeteksi: ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']
Jumlah kelas: 4


# Muat Model Baseline Pre-trained

Muat Muat 'badan' EfficientNet dari KerasCV dengan bobot ImageNET

In [ ]:
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3)
)

24274472/24274472 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Bekukan 'badan' model

In [ ]:
base_model.trainable = False

Bangun model lengkap dengan kepala klasifikasi baru

In [ ]:
inputs = tf.keras.layers.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False) # Penting
x = tf.keras.layers.GlobalAveragePooling2D(name="pooling_layer")(x)
x = tf.keras.layers.Dropout(0.2, name="dropout_layer")(x)
outputs = tf.keras.layers.Dense(num_classes, activation="softmax", name="output_layer")(x)
model = tf.keras.Model(inputs, outputs)

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetv2-b0 (Functional)  │ (None, 7, 7, 1280)     │     5,919,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pooling_layer                   │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_layer (Dropout)         │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, 4)              │         5,124 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,924,436 (22.60 MB)

 Trainable params: 5,124 (20.02 KB)

 Non-trainable params: 5,919,312 (22.58 MB)

# Compile Model Baseline

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Latih Model Baseline

Jumlah Epoch

In [ ]:
EPOCHS = 10

Mulai training

In [ ]:
history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=validation_dataset
)

Epoch 1/10


# Evaluasi model pada data test

In [ ]:
loss, accuracy = model.evaluate(test_dataset)

print("\n==============================================")
print(f"Akurasi Model pada Data Test: {accuracy * 100:.2f}%")
print(f"Loss Model pada Data Test: {loss:.4f}")
print("==============================================")

# Visualisasi Hasil Training Base-Line Model

ambil data dari history

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(len(acc))

Buat dan tampilkan grafik

In [ ]:
# akurasi
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

In [ ]:
# loss
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Buat prediksi dan Confusion Matrix Baseline model

Dapatkan label asli dari test_dataset

In [ ]:
y_true = []
for images, labels in test_dataset:
  y_true.extend(np.argmax(labels.numpy(), axis=1))

Lakukan Prediksi pada test_dataset

In [ ]:
y_pred_probs = model.predict(test_dataset)
y_pred = np.argmax(y_pred_probs, axis=1)

buat confusion matrix

In [ ]:
cm = confusion_matrix(y_true, y_pred)

Tampilkan confusion matrix dengan heatmap

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix for Baseline Model')
plt.show()

Tampilkan classification report untuk metrik lebih detail

In [ ]:
print("\nClassification Report:\n")
print(classification_report(y_true, y_pred, target_names=class_names))

# Analisis Kesalahan Spesifik Baseline Model

Ambil gambar, label asli, dan label prediksi dari test_dataset

In [ ]:
images = []
labels = []
for img, lbl in test_dataset.unbatch(): #lihat satu per satu
    images.append(img.numpy())
    labels.append(np.argmax(lbl.numpy()))

images = np.array(images)
labels = np.array(labels)

Cari indeks dari prediksi yang salah

In [ ]:
misclassified_indices = np.where(y_pred != labels)[0]

Tampilkan beberapa contoh gambar yang salah prediksi

In [ ]:
plt.figure(figsize=(15, 5))
for i, idx in enumerate(misclassified_indices[:5]): # Tampilkan 5 contoh pertama
    plt.subplot(1, 5, i + 1)
    plt.imshow(images[idx].astype('uint8'))
    plt.title(f"True: {class_names[labels[idx]]}\nPred: {class_names[y_pred[idx]]}")
    plt.axis('off')

plt.suptitle('Contoh Gambar yang Salah Klasifikasi', fontsize=16)
plt.show()

# Persiapan Model untuk Fine-Tuning

Cairkan base model

In [ ]:
base_model.trainable = True

print(f"Jumlah lapisand di dalam base model: {len(base_model.layers)}")

Bekukan lapisan awal (agar model tidak dilatih ulang dari awal)

In [ ]:
freeze_until_layer = 100

for layer in base_model.layers[:freeze_until_layer]:
  layer.trainable = False

model.summary()

Compile ulang model dengan learning rate jauh lebih kecil

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

Latih model yang di fine-tune

In [ ]:
FINE_TUNE_EPOCHS = 10
TOTAL_EPOCHS = EPOCHS + FINE_TUNE_EPOCHS

In [ ]:
history_fine_tune = model.fit(
    train_dataset,
    epochs=TOTAL_EPOCHS,
    initial_epoch=history.epoch[-1],
    validation_data=validation_dataset
)

# Evaluasi Model Fine-Tune

In [ ]:
loss, accuracy = model.evaluate(test_dataset)

print("\n==============================================")
print(f"Akurasi Fine-Tune Model pada Data Test: {accuracy * 100:.2f}%")
print(f"Loss Fine-Tune Model pada Data Test: {loss:.4f}")
print("==============================================")

# Visualisasi Hasil Training Fine-Tuned Model

In [ ]:
acc = history_fine_tune.history['accuracy']
val_acc = history_fine_tune.history['val_accuracy']
loss = history_fine_tune.history['loss']
val_loss = history_fine_tune.history['val_loss']
epochs_range = range(len(acc))

In [ ]:
# akurasi
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

In [ ]:
# loss
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Buat prediksi dan Confusion Matrix Fine-Tuned model

In [ ]:
y_true = []
for images, labels in test_dataset:
  y_true.extend(np.argmax(labels.numpy(), axis=1))

y_pred_probs = model.predict(test_dataset)
y_pred = np.argmax(y_pred_probs, axis=1)

cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix for Baseline Model')
plt.show()

In [ ]:
print("\nClassification Report:\n")
print(classification_report(y_true, y_pred, target_names=class_names))

# Analisis kesalahan spesifik Fine-Tuned Model

In [ ]:
images = []
labels = []
for img, lbl in test_dataset.unbatch(): #lihat satu per satu
    images.append(img.numpy())
    labels.append(np.argmax(lbl.numpy()))

images = np.array(images)
labels = np.array(labels)

misclassified_indices = np.where(y_pred != labels)[0]

plt.figure(figsize=(15, 5))
for i, idx in enumerate(misclassified_indices[:5]): # Tampilkan 5 contoh pertama
    plt.subplot(1, 5, i + 1)
    plt.imshow(images[idx].astype('uint8'))
    plt.title(f"True: {class_names[labels[idx]]}\nPred: {class_names[y_pred[idx]]}")
    plt.axis('off')

plt.suptitle('Contoh Gambar yang Salah Klasifikasi', fontsize=16)
plt.show()

# Impor library untuk XAI

In [ ]:
!pip install lime -q

In [ ]:
import numpy as np
import tensorflow as tf
from IPython.display import Image, display, Markdown
import matplotlib.cm as cm
import matplotlib.pyplot as plt

# Analisis XAI dengan Grad-CAM

In [ ]:
# --- RAKIT ULANG ARSITEKTUR (VERSI FINAL YANG BENAR) ---

# 1. Buat kembali base model
base_model_fresh_v2 = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3)
)

# 2. Buat kembali kepala klasifikasi
inputs_fresh_v2 = tf.keras.layers.Input(shape=(224, 224, 3), name="input_layer_fresh_v2")

# --- PERBAIKAN DI SINI: HAPUS `training=False` ---
x_fresh_v2 = base_model_fresh_v2(inputs_fresh_v2) # <-- TIDAK ADA `training=False`

x_fresh_v2 = tf.keras.layers.GlobalAveragePooling2D(name="pooling_layer_fresh_v2")(x_fresh_v2)
x_fresh_v2 = tf.keras.layers.Dropout(0.2, name="dropout_layer_fresh_v2")(x_fresh_v2)
outputs_fresh_v2 = tf.keras.layers.Dense(num_classes, activation="softmax", name="output_layer_fresh_v2")(x_fresh_v2)

# 3. Gabungkan menjadi model yang benar-benar baru
final_model = tf.keras.Model(inputs_fresh_v2, outputs_fresh_v2)

print("Arsitektur model final yang benar berhasil dibuat.")

In [ ]:
# Muat bobot yang sudah di-fine-tune ke dalam model yang baru dan benar
final_model.load_weights("brain_tumor_final_weights.weights.h5")
print("Bobot berhasil ditransplantasikan ke model final.")

fungsi pembantu

In [ ]:
# ==============================================================================
# BLOK KODE GRAD-CAM PENGGANTI (VARIAN TARGET LAYER SPESIFIK)
# ==============================================================================
import tensorflow as tf
import matplotlib.cm as cm
import numpy as np
from IPython.display import Image, display

def make_gradcam_heatmap(img_array, model, last_conv_layer_name="top_activation", pred_index=None):
    """Grad-CAM dengan menarget layer internal pada base model."""
    img_tensor = tf.convert_to_tensor(img_array, dtype=tf.float32)

    # Ambil nested base model (EfficientNetV2B0) dari model
    base = next(l for l in model.layers if isinstance(l, tf.keras.Model))

    # Ekstraktor feature map dari layer internal
    extractor = tf.keras.Model(base.input, base.get_layer(last_conv_layer_name).output)

    with tf.GradientTape() as tape:
        conv_outputs = extractor(img_tensor, training=False)  # [B, H, W, C]
        preds = model(img_tensor, training=False)             # [B, num_classes]
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / (tf.reduce_max(heatmap) + 1e-10)
    return heatmap.numpy()

def superimpose_gradcam(img_path, heatmap, alpha=0.4):
    img = tf.keras.utils.load_img(img_path)
    img = tf.keras.utils.img_to_array(img)

    heatmap = np.uint8(255 * heatmap)
    jet = cm.get_cmap("jet")
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    jet_heatmap = tf.keras.utils.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = tf.keras.utils.img_to_array(jet_heatmap)

    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = tf.keras.utils.array_to_img(superimposed_img)

    return superimposed_img


Hasilkan heatmap Grad-CAM

In [ ]:
!pip install tf-keras-vis matplotlib -q

In [ ]:
# ==============================================================================
# PENDEKATAN FINAL: MENGGUNAKAN LIBRARY TF-KERAS-VIS
# ==============================================================================

# Langkah 1: Instal library (jika belum)
# !pip install tf-keras-vis matplotlib -q

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tf_keras_vis.gradcam import Gradcam
from tf_keras_vis.utils.model_modifiers import ReplaceToLinear
from tf_keras_vis.utils.scores import CategoricalScore
import matplotlib.pyplot as plt

# Langkah 2: Buat instance Gradcam dengan model final yang sudah benar
gradcam = Gradcam(final_model,  # <-- GUNAKAN final_model
                  model_modifier=ReplaceToLinear(),
                  clone=False)

print("--- ANALISIS GRAD-CAM (DENGAN MODEL FINAL YANG BENAR) ---")

# --- CONTOH UNTUK 5 PREDIKSI BENAR ---
print("\n--- 5 Prediksi Benar ---")
for i, idx in enumerate(correct_indices[:5]):
    img_path = test_image_paths[idx]
    img = load_img(img_path, target_size=IMG_SIZE)
    img_array = img_to_array(img)

    predicted_class_index = y_pred[idx]
    score = CategoricalScore([predicted_class_index])

    cam = gradcam(score, img_array, penultimate_layer=-1)

    plt.figure(figsize=(6, 6))
    heatmap = np.uint8(plt.cm.jet(cam[0])[..., :3] * 255)
    plt.imshow(img)
    plt.imshow(heatmap, cmap='jet', alpha=0.5)
    plt.title(f"Asli: {class_names[y_true[idx]]}\nPrediksi: {class_names[y_pred[idx]]}")
    plt.axis('off')
    plt.show()

# (Sisa kode untuk prediksi salah bisa ditambahkan jika perlu)

Analisis LIME

In [ ]:
!pip install lime -q

In [ ]:
# KODE LIME PERBAIKAN
import lime
from lime import lime_image
from skimage.segmentation import mark_boundaries
import matplotlib.pyplot as plt

# Buat explainer LIME
explainer = lime_image.LimeImageExplainer()

# Fungsi untuk melakukan prediksi pada gambar yang dimodifikasi oleh LIME
def predict_fn(images):
    return model.predict(images)

# Kita akan gunakan gambar yang sama dari Grad-CAM agar bisa dibandingkan
selected_indices = np.concatenate([correct_indices[:5], misclassified_indices[:5]])

print("\n\n--- ANALISIS LIME ---")

# Loop melalui gambar yang sama
for i, idx in enumerate(selected_indices):
    img_path = test_image_paths[idx]

    # LIME butuh gambar dalam format array numpy
    img = tf.keras.utils.load_img(img_path, target_size=IMG_SIZE)
    img_array = tf.keras.utils.img_to_array(img)

    # Dapatkan penjelasan dari LIME
    explanation = explainer.explain_instance(
        img_array.astype('float32'),  # <-- PERBAIKAN 1
        predict_fn,
        top_labels=1,
        hide_color=0,
        num_samples=1000 # Jumlah sampel untuk diuji
    )

    # Tampilkan hasil LIME
    print(f"\nGambar {i+1}: Label Asli = {class_names[y_true[idx]]}, Prediksi = {class_names[y_pred[idx]]}")
    temp, mask = explanation.get_image_and_mask(
        explanation.top_labels[0],
        positive_only=True,
        num_features=5,
        hide_rest=False
    )
    # Normalisasi 'temp' ke rentang [0,1] untuk plotting yang benar
    plt.imshow(mark_boundaries(temp / 255.0, mask)) # <-- PERBAIKAN 2
    plt.axis('off')
    plt.show()